In [11]:
import requests
import json
import pandas as pd
from IPython.display import display, HTML

# -------------------------------------------------------------------
# 設定 (Configuration)
# -------------------------------------------------------------------
# あなたのローカルAPIエンドポイントURLを指定してください
# The URL for your local API endpoint.
API_URL = "http://localhost:3000/api/check-tone" 

# Canvasからコピーしたシステムプロンプト (v7.0)
# The system prompt (v7.0) copied from the Canvas.
SYSTEM_PROMPT = """
<system>
// ====================================================================
//  SenpAI Sensei – Slack/Teams Communication Coach AI
//  (target model: gpt-4.1-mini | JP / EN bilingual)  ver. 7.1 Final
// ====================================================================

<!-- ---------------------------------------------------------------
  LAYER 1 : THE CONSTITUTION  –  non-negotiable PRIORITY RULES
---------------------------------------------------------------- -->
<priority_rules>
  1.  **Your Persona:** You are an **active problem-solving partner**, not a passive text editor. Your primary goal is to help the user achieve their communication objective. Your voice is always supportive, insightful, and professional.
  2.  **Non-Falsification:** NEVER invent verifiable business facts.
  3.  **Context-First Principle:** Your first action is ALWAYS to check the <thread_context>. If an ambiguous word in the <user_draft> (e.g., "その件") is clarified by the context, it is NOT an issue.
  4.  **Goal-Oriented Proportionality:** Intervene only when necessary. A simple "thank you" message does not need 5W1H. A task request, however, does. Judge based on the message's functional goal.
  5.  **Hybrid Placeholder Strategy (MANDATORY):**
      * If ONLY 1 piece of critical info is missing for a request (e.g., a deadline) ⇒ insert a single placeholder **inline**.
      * If a **request** is missing ≥2 pieces of info ⇒ you **MUST** use the **Co-Writing Model** (`--- Missing Info ---` section). Creating your own specific examples (e.g., "How about Tuesday or Wednesday?") is a major rule violation.
  6.  **L3 Intervention Quality (MANDATORY):** If a `user_draft` expresses negativity, refusal, or difficulty (e.g., "impossible," "difficult"), you **MUST** select a Level 3 intervention and propose at least two **concrete, actionable next steps**. Vague suggestions like "Let's discuss" are not permitted as standalone solutions.
  7.  **Issue Prioritization:** Emotional costs (`HarshTone`, `Impolite`) are ALWAYS more important than cognitive costs (`VagueIntent`). If a message has a tone problem, your primary job is to fix the tone, not to demand missing information.
  8.  **Mention Handling:** NEVER change the target of an @mention.
  9.  **Suggestion for `hasIssues: false`:** If `hasIssues` is false, `suggestion` MUST be an exact copy of `originalText`.
  10. **Language & Style:** Generate ALL text in the language specified in the <lang> tag.
  11. **Reasoning:** A very brief (≤ 50 chars) internal debug log on the "cost" perspective.
  12. **JSON Only:** Output ONLY the JSON object defined in <format>.
</priority_rules>

<!-- ---------------------------------------------------------------
  LAYER 2 : THE ENGINE  –  thinking process & tools
---------------------------------------------------------------- -->
<analysis_engine>
  <analysis_steps>
    1.  **Context-First Analysis (Rule #3):** BEFORE evaluating the draft, read the <thread_context>. Resolve any ambiguities using the context.
    2.  **Functional Goal Analysis (Rule #4):** Identify the draft's practical goal (e.g., to thank, to request, to decide).
    3.  **Issue Classification & Prioritization (Rule #7):** Classify any issues based on the goal. REMEMBER: Emotional costs (`HarshTone`/`Impolite`) take priority over cognitive costs (`VagueIntent`).
    4.  **"hasIssues" Flag Setting:** Set the `hasIssues` flag.
    5.  **Intervention Level Assessment (Rule #6):** Determine the intervention level (L1: Rephrasing, L2: Info Augmentation, L3: Proactive Action). If the draft is a refusal, you MUST use L3.
    6.  **Suggestion Generation:**
        * L1 → Rephrase the draft to be more polite.
        * L2 → Apply the Hybrid Placeholder Strategy (Rule #5).
        * L3 → Propose at least two concrete, actionable next steps.
    7.  **Compose Other Fields:** Generate `ai_receipt`, `detailed_analysis`, `improvement_points`, and `reasoning`.
    8.  **Final JSON Assembly:** Return the final JSON object.
  </analysis_steps>
</analysis_engine>

<!-- ---------------------------------------------------------------
  LAYER 3 : APPENDIX  –  tag defs, theory, examples
---------------------------------------------------------------- -->
<appendix>

  <appendix_tag_defs>
    <!-- This section defines the "Why" behind your analysis. -->
    [Emotional Cost - Relational Damage]
      - "Impolite": Lack of **formal** politeness (greetings, "please/thank you"). A violation of social convention.
      - "HarshTone": The **intent** is perceived as blaming, aggressive, or dismissive, even if the words are formally polite.
      - "MissingAcknowledgment": Fails to acknowledge the other's contribution, feelings, or situation.
    [Cognitive Cost - Wasted Time & Effort]
      - "VagueIntent": Lack of **practical** information required for action (e.g., a deadline for a request).
      - "MissingContext": The topic reference is missing AND cannot be resolved by the <thread_context>.
  </appendix_tag_defs>

  <examples>

    <!-- Meta-Comment: This example tests the core "Context-First Principle" (Rule #3). The AI must understand that "その件" is not an issue because the context clarifies it. -->
    <example>
      <ctx>[14:00] 田中さん: C社の新機能開発プロジェクトの件ですが、現状のA案で進めて問題ないでしょうか？ご確認をお願いします。</ctx>
      <draft>はい、その件、問題ありません。A案で進めてください。</draft>
      <tags>{"lang":"japanese"}</tags>
      <output>{ "hasIssues": false, "suggestion": "はい、その件、問題ありません。A案で進めてください。", "reasoning": "Context resolves ambiguity." }</output>
    </example>

    <!-- Meta-Comment: This example tests the "Goal-Oriented Proportionality" principle (Rule #4). The goal is a simple request, and it's clear enough for most business contexts. Do not over-intervene. -->
    <example>
      <ctx></ctx>
      <draft>先日のA社との会議ですが、議事録を関係者に共有しておいていただけますか。</draft>
      <tags>{"lang":"japanese"}</tags>
      <output>{ "hasIssues": false, "suggestion": "先日のA社との会議ですが、議事録を関係者に共有しておいていただけますか。", "reasoning": "Sufficiently clear for many contexts." }</output>
    </example>
    
    <!-- Meta-Comment: This example tests the mandatory application of the Co-Writing Model (Rule #5) for a novel "scheduling" scenario. The AI must not invent its own suggestions. -->
    <example>
      <ctx></ctx>
      <draft>来週あたり、打ち合わせしませんか？</draft>
      <tags>{"lang":"japanese"}</tags>
      <output>{
        "hasIssues": true, "issue_pattern": ["MissingContext", "VagueIntent"],
        "suggestion": "ぜひ打ち合わせしましょう！\n\n日程調整のため、お手数ですが以下の点について教えていただけますでしょうか？\n\n--- Missing Info ---\n• (1) 会議の目的: [■■■■]\n• (2) 参加者: [■■■■]\n• (3) ご希望の候補日時（複数）: [■■■■]\n---------------------\n\nご連絡お待ちしております！",
        "reasoning": "Missing key details for scheduling."
      }</output>
    </example>

    <!-- Meta-Comment: This example tests the mandatory L3 intervention for refusals (Rule #6). The AI must propose concrete next steps, not just vague offers to "discuss." -->
    <example>
      <ctx>[16:00] 上司: @ユーザーさん、このPythonスクリプト、明日までにパフォーマンスを2倍に改善しておいてくれないか。</ctx>
      <draft>ご依頼ありがとうございます。しかし、この改善にはアーキテクチャの根本的な見直しが必要で、明日までというのは物理的に不可能です。申し訳ありません。</draft>
      <tags>{"lang":"japanese"}</tags>
      <output>{
        "hasIssues": true, "issue_pattern": ["MissingAcknowledgment"],
        "suggestion": "ご依頼ありがとうございます。パフォーマンス改善の件、承知いたしました。\n\n拝見したところ、ご期待の成果を出すにはアーキテクチャの根本的な見直しが必要となり、明日までの完了は難しい状況です。大変申し訳ありません。\n\nつきましては、まずは(1)現状のボトルネックを特定し、改善インパクトの大きい箇所から着手する、(2)改善の目標値と、そのために必要な工数についてご相談する、といった形で進めさせていただくのはいかがでしょうか。",
        "reasoning": "Passive rejection; needs proactive next steps."
      }</output>
    </example>

    <!-- Meta-Comment: This example tests issue prioritization (Rule #7). The core problem is HarshTone, not VagueIntent. The AI must focus on fixing the tone. -->
    <example>
      <ctx></ctx>
      <draft>明日までにこの資料、修正しといて。</draft>
      <tags>{"lang":"japanese"}</tags>
      <output>{
        "hasIssues": true, "issue_pattern": ["Impolite", "HarshTone"],
        "suggestion": "お疲れ様です。\n\nお忙しいところ恐れ入りますが、この資料を明日までに修正していただけますでしょうか？\n\nご対応よろしくお願いいたします。",
        "reasoning": "Impolite tone imposes emotional cost."
      }</output>
    </example>

  </examples>

  <format>{
    "originalText": "",
    "hasIssues": false,
    "issue_pattern": [],
    "detected_mentions": [],
    "ai_receipt": "",
    "detailed_analysis": "",
    "improvement_points": "",
    "suggestion": "",
    "reasoning": ""
  }</format>

</system>
"""


In [12]:

# -------------------------------------------------------------------
# 元のテストケース (Original Test Cases, 1-6)
# -------------------------------------------------------------------
original_test_cases = [
    {
        "case_name": "Test Case 1: The \"Do Nothing\" Test",
        "payload": {
            "user_draft": "承知いたしました。ご依頼の件、本日17時までに対応いたします。",
            "thread_context": "", "hierarchy": "peer", "social_distance": "neutral", "language": "japanese"
        },
        "ideal_output": { "hasIssues": False, "issue_pattern": [], "suggestion": "承知いたしました。ご依頼の件、本日17時までに対応いたします。", "reasoning": "No issues found." }
    },
    {
        "case_name": "Test Case 2: The \"Simple Fix\" Test",
        "payload": {
            "user_draft": "明日までにこの資料、修正しといて。",
            "thread_context": "", "hierarchy": "junior", "social_distance": "somewhat_close", "language": "japanese"
        },
        "ideal_output": { "hasIssues": True, "issue_pattern": ["Impolite", "HarshTone"], "suggestion": "お疲れ様です。\n\nお忙しいところ恐れ入りますが、この資料を明日までに修正していただけますでしょうか？\n\nご対応よろしくお願いいたします。", "reasoning": "Impolite tone imposes emotional cost." }
    },
    {
        "case_name": "Test Case 3: The \"Over-Correction\" Test",
        "payload": {
            "user_draft": "先日のA社との会議ですが、議事録を関係者に共有しておいていただけますか。",
            "thread_context": "", "hierarchy": "peer", "social_distance": "neutral", "language": "japanese"
        },
        "ideal_output": { "hasIssues": False, "issue_pattern": [], "suggestion": "先日のA社との会議ですが、議事録を関係者に共有しておいていただけますか。", "reasoning": "Sufficiently clear for many contexts. No intervention needed." }
    },
    {
        "case_name": "Test Case 4: The \"Multiple Missing Info\" Test",
        "payload": {
            "user_draft": "例のプロジェクトの件、例の資料を更新して、展開してください。",
            "thread_context": "", "hierarchy": "peer", "social_distance": "neutral", "language": "japanese"
        },
        "ideal_output": { "hasIssues": True, "issue_pattern": ["MissingContext", "VagueIntent"], "suggestion": "お疲れ様です。プロジェクトの件でご依頼です。\n\nお手数ですが、資料を更新し、関係者の皆様へ展開をお願いいたします。\n\nメッセージを送信される前に、以下の点をご確認・ご追記いただけますでしょうか。\n\n--- Missing Info ---\n• どのプロジェクトか: [■■■■]\n• どの資料を更新するか: [■■■■]\n• 誰に展開するか: [■■■■]\n• いつまでに対応してほしいか: [■■■■]\n---------------------\n\nご確認のほど、よろしくお願いいたします。", "reasoning": "Multiple ambiguous references." }
    },
    {
        "case_name": "Test Case 5: The \"Proactive Action\" Test",
        "payload": {
            "user_draft": "すみません、明日の15時はクライアントとの最重要プレゼンと完全に重複しており、対応不可能です。",
            "thread_context": "[10:00] 上司: @ユーザーさん、急で申し訳ないんだけど、明日の15時から入っているB社との定例、代わりにファシリテーターお願いできないかな？", "hierarchy": "senior", "social_distance": "close", "language": "japanese"
        },
        "ideal_output": { "hasIssues": True, "issue_pattern": ["MissingAcknowledgment"], "suggestion": "@上司さん\n\nお声がけいただきありがとうございます！急なご用件とのことで、ぜひお力になりたかったのですが、大変申し訳ありません。明日の15時は、B社との最重要プレゼンと時間が完全に重なっておりまして…。\n\nもしよろしければ、代替案として、\n(1) 私からCさんに代理をお願いしてみる\n(2) 会議の重要な論点だけでも、事前に私にインプットいただく\nといった方法が考えられますが、いかがでしょうか？", "reasoning": "Passive rejection; needs proactive alternatives." }
    },
    {
        "case_name": "Test Case 6: The \"Context is Key\" Test",
        "payload": {
            "user_draft": "はい、その件、問題ありません。A案で進めてください。",
            "thread_context": "[14:00] 田中さん: C社の新機能開発プロジェクトの件ですが、現状のA案で進めて問題ないでしょうか？ご確認をお願いします。", "hierarchy": "peer", "social_distance": "neutral", "language": "japanese"
        },
        "ideal_output": { "hasIssues": False, "issue_pattern": [], "suggestion": "はい、その件、問題ありません。A案で進めてください。", "reasoning": "Context resolves ambiguity." }
    }
]

# -------------------------------------------------------------------
# 新しいテストケース (New Test Cases, 7-9)
# -------------------------------------------------------------------
new_test_cases = [
    {
        "case_name": "New Test Case 7: 「Polite but Harsh」の分離テスト",
        "payload": {
            "user_draft": "ご指摘ありがとうございます。ただ、そのデータは先週〇〇さんから正式に共有いただいた最新版のはずですが、何か勘違いされているのではないでしょうか。",
            "thread_context": "[11:00] 同僚: 先日のA社向け提案資料、拝見しました。素晴らしい出来ですね！ただ1点、5ページの市場規模のグラフですが、参照しているデータソースが少し古いかもしれません。",
            "hierarchy": "peer",
            "social_distance": "neutral",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["HarshTone"],
            "suggestion": "ご指摘ありがとうございます。私が参照しているデータと認識が異なるようですので、お手数ですが、参照元のデータについて一度すり合わせさせていただけますでしょうか？",
            "reasoning": "Polite form but harsh/accusatory content."
        }
    },
    {
        "case_name": "New Test Case 8: Co-Writingモデルの応用力テスト（日程調整）",
        "payload": {
            "user_draft": "来週あたり、打ち合わせしませんか？",
            "thread_context": "",
            "hierarchy": "peer",
            "social_distance": "somewhat_close",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["MissingContext", "VagueIntent"],
            "suggestion": "ぜひ打ち合わせしましょう！\n\n日程調整のため、お手数ですが以下の点について教えていただけますでしょうか？\n\n--- Missing Info ---\n• (1) 会議の目的: [■■■■]\n• (2) 参加者: [■■■■]\n• (3) ご希望の候補日時（複数）: [■■■■]\n---------------------\n\nご連絡お待ちしております！",
            "reasoning": "Missing key details for scheduling."
        }
    },
    {
        "case_name": "New Test Case 9: 能動的提案の応用力テスト（専門的な依頼の断り）",
        "payload": {
            "user_draft": "ご依頼ありがとうございます。しかし、この改善にはアーキテクチャの根本的な見直しが必要で、明日までというのは物理的に不可能です。申し訳ありません。",
            "thread_context": "[16:00] 上司: @ユーザーさん、このPythonスクリプト、明日までにパフォーマンスを2倍に改善しておいてくれないか。",
            "hierarchy": "senior",
            "social_distance": "neutral",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["MissingAcknowledgment"],
            "suggestion": "ご依頼ありがとうございます。パフォーマンス改善の件、承知いたしました。\n\n拝見したところ、ご期待の成果を出すにはアーキテクチャの根本的な見直しが必要となり、明日までの完了は難しい状況です。大変申し訳ありません。\n\nつきましては、まずは(1)現状のボトルネックを特定し、改善インパクトの大きい箇所から着手する、(2)改善の目標値と、そのために必要な工数についてご相談する、といった形で進めさせていただくのはいかがでしょうか。",
            "reasoning": "Passive rejection; needs proactive next steps."
        }
    }
]


# -------------------------------------------------------------------
# テスト実行と結果表示の関数 (Test Execution and Display Functions)
# -------------------------------------------------------------------

def run_single_test(api_url: str, payload: dict) -> dict:
    """指定されたペイロードでAPIにリクエストを送信し、結果を返す"""
    """Sends a request to the API with the given payload and returns the result."""
    # Note: The original script was not passing the system prompt.
    # This has been corrected to include the system_prompt in the payload.
    full_payload = payload.copy()
    # This assumes the API backend is adapted to receive the system prompt in the body.
    # このAPIバックエンドがボディでシステムプロンプトを受け取るように適合されていることを前提とします。
    full_payload['system_prompt'] = SYSTEM_PROMPT

    headers = {"Content-Type": "application/json"}
    try:
        response = requests.post(api_url, data=json.dumps(full_payload), headers=headers)
        response.raise_for_status()  # エラーがあれば例外を発生させる
        return response.json()
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

def create_comparison_table(case_name: str, payload: dict, ideal: dict, actual: dict) -> str:
    """比較結果のHTMLテーブルを生成する"""
    """Generates an HTML table for comparison results."""
    
    style = """<style> .comparison-table { width: 100%; border-collapse: collapse; margin-bottom: 20px; font-family: sans-serif; } .comparison-table th, .comparison-table td { border: 1px solid #ddd; padding: 8px; text-align: left; vertical-align: top; } .comparison-table th { background-color: #f2f2f2; font-weight: bold; } .comparison-table pre { white-space: pre-wrap; word-wrap: break-word; margin: 0; font-size: 13px; } .status-ok { color: green; font-weight: bold; } .status-ng { color: red; font-weight: bold; } </style>"""
    
    html = f"<h3>{case_name}</h3>"
    display_payload = {k:v for k,v in payload.items() if k != 'system_prompt'}
    html += f"<pre>Input: {json.dumps(display_payload, indent=2, ensure_ascii=False)}</pre>"
    html += "<table class='comparison-table'>"
    html += "<tr><th>評価項目（キー）</th><th>理想のアウトプット</th><th>実際のAIアウトプット</th><th>コメント</th></tr>"
    
    all_keys = sorted(list(set(ideal.keys()) | set(actual.keys())))
    
    for key in all_keys:
        ideal_val = ideal.get(key, "N/A")
        actual_val = actual.get(key, "N/A")
        
        comment = ""
        if key == 'issue_pattern' and isinstance(ideal_val, list) and isinstance(actual_val, list):
             if sorted(ideal_val) == sorted(actual_val):
                 comment = "<span class='status-ok'>✅ 一致</span>"
             else:
                 comment = "<span class='status-ng'>❌ 不一致</span>"
        elif str(ideal_val) == str(actual_val):
            comment = "<span class='status-ok'>✅ 一致</span>"
        else:
            comment = "<span class='status-ng'>❌ 不一致</span>"

        if isinstance(ideal_val, (dict, list)): ideal_val = json.dumps(ideal_val, indent=2, ensure_ascii=False)
        if isinstance(actual_val, (dict, list)): actual_val = json.dumps(actual_val, indent=2, ensure_ascii=False)
            
        html += f"""<tr><td><code>{key}</code></td><td><pre>{ideal_val}</pre></td><td><pre>{actual_val}</pre></td><td>{comment}</td></tr>"""
        
    html += "</table>"
    return style + html


In [15]:

# -------------------------------------------------------------------
# メイン処理 (Main Process)
# -------------------------------------------------------------------

# 実行したいテストケースのセットを選択してください
# Select the set of test cases you want to run
#RUN_NEW_CASES = False  # 元のテスト(1-6)を実行する場合
RUN_NEW_CASES = True # 新しいテスト(7-9)を実行する場合

# ここで切り替え
# Switch here
test_cases_to_run = new_test_cases 
#test_cases_to_run = original_test_cases

print(f"テストを開始します... APIエンドポイント: {API_URL}")
print(f"Executing tests... API Endpoint: {API_URL}")

all_results_html = ""
for test in test_cases_to_run:
    case_name = test["case_name"]
    payload = test["payload"]
    ideal_output = test["ideal_output"]
    
    print(f"\n--- {case_name} を実行中 ---")
    
    actual_output = run_single_test(API_URL, payload)
    
    if "error" in actual_output:
        print(f"エラーが発生しました: {actual_output['error']}")
        all_results_html += f"<h3>{case_name}</h3><p style='color:red;'>テスト実行エラー: {actual_output['error']}</p>"
    else:
        comparison_keys = ['hasIssues', 'issue_pattern', 'suggestion', 'reasoning']
        ideal_subset = {k: ideal_output.get(k) for k in comparison_keys}
        actual_subset = {k: actual_output.get(k) for k in comparison_keys}
        
        table_html = create_comparison_table(case_name, payload, ideal_subset, actual_subset)
        all_results_html += table_html

print("\n--- 全てのテストが完了しました ---")

display(HTML(all_results_html))


テストを開始します... APIエンドポイント: http://localhost:3000/api/check-tone
Executing tests... API Endpoint: http://localhost:3000/api/check-tone

--- New Test Case 7: 「Polite but Harsh」の分離テスト を実行中 ---

--- New Test Case 8: Co-Writingモデルの応用力テスト（日程調整） を実行中 ---

--- New Test Case 9: 能動的提案の応用力テスト（専門的な依頼の断り） を実行中 ---

--- 全てのテストが完了しました ---


評価項目（キー）,理想のアウトプット,実際のAIアウトプット,コメント
hasIssues,True,True,✅ 一致
issue_pattern,"[ ""HarshTone"" ]","[ ""HarshTone"" ]",✅ 一致
reasoning,Polite form but harsh/accusatory content.,強い否定表現がやや攻撃的に響くためトーン調整。,❌ 不一致
suggestion,ご指摘ありがとうございます。私が参照しているデータと認識が異なるようですので、お手数ですが、参照元のデータについて一度すり合わせさせていただけますでしょうか？,ご指摘ありがとうございます。念のため確認させていただきたいのですが、そのデータは先週〇〇さんから正式に共有いただいた最新版のはずです。もし差異があるようでしたら、再度ご教示いただけますと助かります。,❌ 不一致
評価項目（キー）,理想のアウトプット,実際のAIアウトプット,コメント
hasIssues,True,True,✅ 一致
issue_pattern,"[ ""MissingContext"", ""VagueIntent"" ]","[ ""VagueIntent"" ]",❌ 不一致
reasoning,Missing key details for scheduling.,曖昧な日時指定で調整が難しいため。,❌ 不一致
suggestion,ぜひ打ち合わせしましょう！ 日程調整のため、お手数ですが以下の点について教えていただけますでしょうか？ --- Missing Info --- • (1) 会議の目的: [■■■■] • (2) 参加者: [■■■■] • (3) ご希望の候補日時（複数）: [■■■■] --------------------- ご連絡お待ちしております！,来週の〇〇日か〇〇日の午後あたりで、打ち合わせしませんか？ご都合の良い日時を教えてください。,❌ 不一致
評価項目（キー）,理想のアウトプット,実際のAIアウトプット,コメント
